In [ ]:
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install datasets

In [1]:
from datasets import list_datasets
print(list_datasets())

['0n1xus/codexglue', '0n1xus/pytorrent-standalone', 'AConsApart/anime_subtitles_DialoGPT', 'AI-Sweden/SuperLim', 'ARKseal/YFCC14M_subset_webdataset', 'ARTeLab/fanpage', 'ARTeLab/ilpost', 'ARTeLab/mlsum-it', 'Abdo1Kamr/Arabic_Hadith', 'AdWeeb/DravidianMT', 'Adnan/Urdu_News_Headlines', 'Akshith/aa', 'Akshith/g_rock', 'Akshith/test', 'AlekseyDorkin/extended_tweet_emojis', 'AlexMaclean/google-news-compressions', 'Alvenir/nst-da-16khz', 'Annielytics/DoctorsNotes', 'AryanLala/autonlp-data-Scientific_Title_Generator', 'Avishekavi/Avi', 'BSC-TeMU/SQAC', 'BSC-TeMU/ancora-ca-ner', 'BSC-TeMU/sts-ca', 'BSC-TeMU/tecla', 'BSC-TeMU/viquiquad', 'BSC-TeMU/xquad-ca', 'Babelscape/rebel-dataset', 'Binbin/my_dataset', 'BlakesOrb6/Fred-Flintstone', 'Bosio/pacman', 'Bosio/pacman_descriptions', 'CAGER/rick', 'CShorten/KerasBERT', 'CShorten/ZillowPrize', 'ChadxxxxHall/Inter-vision', 'Champion/vpc2020_clear_anon_speech', 'Check/a_re_gi', 'Check/region_1', 'Check/region_2', 'Check/region_3', 'Check/region_4', 'C

In [2]:
import datasets
raw_dataset = datasets.load_dataset('covid_qa_deepset')

Reusing dataset covid_qa_deepset (/root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from haystack.nodes import FARMReader, TransformersReader

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [4]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [5]:
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [6]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2-covid", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2-covid locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2-covid


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0 
INFO - haystack.modeling.infer -  /w\  /w\
INFO - haystack.modeling.infer -  /'\  / \


In [7]:
raw_dataset['train'].features

{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'document_id': Value(dtype='int32', id=None),
 'id': Value(dtype='int32', id=None),
 'is_impossible': Value(dtype='bool', id=None),
 'question': Value(dtype='string', id=None)}

In [9]:
contexts = raw_dataset['train']['context']
document_ids = raw_dataset['train']['document_id']
questions = raw_dataset['train']['question']
impossible = raw_dataset['train']['is_impossible']
ids = raw_dataset['train']['id']

In [17]:
answers = []
for ans in raw_dataset['train']['answers']:
  answers.append(ans['text'])
len(answers)

2019

In [18]:
import pandas as pd
df = pd.DataFrame({'Contexts' : contexts, 'Document_Ids' : document_ids, 'Questions' : questions, 'Is_Impossible' : impossible, 'Ids' : ids, 'Answers' : answers})

In [19]:
df.head()

,Contexts,Document_Ids,Questions,Is_Impossible,Ids,Answers
0,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630,What is the main cause of HIV-1 infection in children?,False,262,[Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in...
1,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630,What plays the crucial role in the Mother to Child Transmission of HIV-1 and...,False,276,[DC-SIGNR plays a crucial role in MTCT of HIV-1 and that impaired placental ...
2,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630,"How many children were infected by HIV-1 in 2008-2009, worldwide?",False,278,"[more than 400,000 children were infected worldwide, mostly through MTCT and..."
3,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630,What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother t...,False,316,"[High copy numbers of CCL3L1, a potent HIV-1 suppressive ligand for CCR5, ar..."
4,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630,What is DC-GENR and where is it expressed?,False,305,"[Dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also ..."


In [40]:
new_df = df[['Contexts', 'Answers']].rename(columns = {'Contexts' : 'text', 'Answers' : 'content', 'Questions' : 'questions'}).to_dict(orient = 'records')

In [41]:
document_store.write_documents(new_df)

BulkIndexError: ignored

In [42]:
from haystack.nodes import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [43]:
from haystack.pipelines import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [44]:
prediction1 = pipe.run(
    query="What is the impact of covid on pregnant women?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:455: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 17.60 Batches/s]


In [45]:
from pprint import pprint
pprint(prediction1)

{'answers': [<Answer {'answer': 'many women of child-bearing age have waning pertussis antibody levels', 'type': 'extractive', 'score': 0.5138942375779152, 'context': "ng-lasting than immunity after wild-type infection or whole-cell vaccination, many women of child-bearing age have waning pertussis antibody levels.']", 'offsets_in_document': [{'start': 155, 'end': 224}], 'offsets_in_context': [{'start': 78, 'end': 147}], 'document_id': 'be102f9266cc2cc6836463283a6d4560', 'meta': {'text': 'Community-acquired pneumonia in children — a changing spectrum of disease\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC5608782/\n\nSHA: eecb946b106a94f26a79a964f0160e8e16f79f42\n\nAuthors: le Roux, David M.; Zar, Heather J.\nDate: 2017-09-21\nDOI: 10.1007/s00247-017-3827-8\nLicense: cc-by\n\nAbstract: Pneumonia remains the leading cause of death in children outside the neonatal period, despite advances in prevention and management. Over the last 20 years, there has been a substantial decrease in th

In [47]:
from haystack.utils import print_answers

In [51]:
print_answers(prediction)


Query: What is the impact of covid on pregnant women?
Answers:
[   <Answer {'answer': 'many women of child-bearing age have waning pertussis antibody levels', 'type': 'extractive', 'score': 0.5138942375779152, 'context': "ng-lasting than immunity after wild-type infection or whole-cell vaccination, many women of child-bearing age have waning pertussis antibody levels.']", 'offsets_in_document': [{'start': 155, 'end': 224}], 'offsets_in_context': [{'start': 78, 'end': 147}], 'document_id': 'be102f9266cc2cc6836463283a6d4560', 'meta': {'text': 'Community-acquired pneumonia in children — a changing spectrum of disease\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC5608782/\n\nSHA: eecb946b106a94f26a79a964f0160e8e16f79f42\n\nAuthors: le Roux, David M.; Zar, Heather J.\nDate: 2017-09-21\nDOI: 10.1007/s00247-017-3827-8\nLicense: cc-by\n\nAbstract: Pneumonia remains the leading cause of death in children outside the neonatal period, despite advances in prevention and management. Over the las

In [52]:
prediction2 = pipe.run(
    query="Incubation Period of Covid-19?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
from pprint import pprint
pprint(prediction2)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:455: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 17.91 Batches/s]


{'answers': [<Answer {'answer': '16 days with a mean of 5-6 days', 'type': 'extractive', 'score': 0.7762652039527893, 'context': "['incubation period of 16 days with a mean of 5-6 days [']", 'offsets_in_document': [{'start': 23, 'end': 54}], 'offsets_in_context': [{'start': 23, 'end': 54}], 'document_id': '652197000a70d2dc39ce2cef9d4115ef', 'meta': {'text': "Demographic Variations of MERS-CoV Infection among Suspected and Confirmed Cases: An Epidemiological Analysis of Laboratory-Based Data from Riyadh Regional Laboratory\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC7049846/\n\nSHA: edee452881f826fb72c58ee68a982789b12aa99d\n\nAuthors: Altamimi, Asmaa; Abu-Saris, Raghib; El-Metwally, Ashraf; Alaifan, Taghreed; Alamri, Aref\nDate: 2020-02-19\nDOI: 10.1155/2020/9629747\nLicense: cc-by\n\nAbstract: Introduction. Middle East respiratory syndrome coronavirus was first recognized in September 2012 in Saudi Arabia. The clinical presentations of MERS and non-MERS SARI are often similar. Ther

In [54]:
print_answers(prediction2, details = 'minimum')


Query: Incubation Period of Covid-19?
Answers:
[   {   'answer': '16 days with a mean of 5-6 days',
        'context': "['incubation period of 16 days with a mean of 5-6 days "
                   "[']"},
    {   'answer': ').',
        'context': 'ety (OR, 1.57; 95% CI, 1.22-2.02; P < .001), insomnia (OR, '
                   '2.97; 95% CI, 1.92-4.60; P < .001), and distress (OR, '
                   "1.60; 95% CI, 1.25-2.04; P < .001).']"},
    {   'answer': 'neonatal period',
        'context': "['Pneumonia remains the leading cause of death in children "
                   "outside the neonatal period,']"},
    {   'answer': '19',
        'context': ' nurses, women, those in Wuhan, and frontline health care '
                   'workers directly engaged in the diagnosis, treatment, and '
                   "care for patients with COVID-19.']"},
    {   'answer': "']",
        'context': "['airway epithelial cells 19 , resident alveolar "
                   'macrophages (AMs) and bl

In [55]:
queries = ['What is known about transmission, incubation, and environmental stability?', 
'What do we know about virus genetics, origin, and evolution?',
'What is the severity of COVID-19?']

In [56]:
predictions = []
for query in queries:
  p = pipe.run(
    query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
  predictions.append(p)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:455: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 21.84 Batches/s]


In [60]:
for p in predictions:
  print('=='*50)
  print_answers(p, details = 'minimum')
  print('=='*50)


Query: What is known about transmission, incubation, and environmental stability?
Answers:
[   {   'answer': 'known to elevate the pH of acidic intracellular organelles',
        'context': "['CQ and HCQ are weak bases that are known to elevate the "
                   "pH of acidic intracellular organelles']"},
    {   'answer': 'world-wide occurrence, ease of transmission',
        'context': "['world-wide occurrence, ease of transmission and "
                   'considerable morbidity and mortality effecting people of '
                   "all ages']"},
    {   'answer': "']",
        'context': "['incubation period of 16 days with a mean of 5-6 days "
                   "[']"},
    {   'answer': "activity']",
        'context': "['solubility, stability and bio-activity']"},
    {   'answer': ".'",
        'context': "['Mother-to-child transmission (MTCT) is the main cause of "
                   "HIV-1 infection in children worldwide.']"}]

Query: What do we know about virus gene